# Function Calling/Tool Use


## Just a quick definition

Function calling in the context of Large Language Models (LLMs) refers to the ability of the model to invoke external functions or tools during its response generation. Instead of only generating text, the LLM can recognize when a task requires external data or computation, call a predefined function with the appropriate arguments, and then use the function's output to continue its response.

Think of having a super smart robot that you can only speak with. Pretty useless at doing anything aside from talking. Now imagine you have given that robot a hammer and some nails. It can now put up that wall painting that's been waiting forever to be hung :D

Except, the way we as AI Engineers give AI tools is by defining python functions and describing that function in detail to the LLM so it knows what tools it has access to, what each tool can do, what are its inputs and expected outputs.

Just remember, a *tool* in its simplest form is just a *python function* that you have defined. It can be as simple as a calculator function or something like being able to call external APIs.

![tool-use](../../images/tool-use.png)

*Image courtesy of [API Deck](https://www.apideck.com/blog/llm-tool-use-and-function-calling)*

## Step 1: Import libraries

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key is missing")

## Step 2: Define a tool

**The tool:** [OpenWeather API](https://openweathermap.org/api/one-call-3)

**API Call:** `https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m`

In [2]:
import requests

# Define the python function
def get_weather(lat, lon):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


In [3]:
# Call the function
get_weather(24.343627, 54.497922)

37.3

## Step 3: Call a chat model normally

In [4]:
# Call OpenAI Chat model

client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "user", "content": "What's the weather like in Dubai?"}
    ]
)

print(response.output_text)

I don't have real-time data access to provide current weather conditions. However, Dubai typically has a desert climate, characterized by hot summers and mild winters. In summer (June to September), temperatures can exceed 40°C (104°F), while winter (December to February) sees milder temperatures, ranging from around 14°C to 24°C (57°F to 75°F) during the day.

For the most accurate and up-to-date weather information, I recommend checking a reliable weather website or app.


In [5]:
# View all outputs
response.__dict__

{'id': 'resp_68a814342a68819cb30f1da5b7723921014223f5b3c50db0',
 'created_at': 1755845684.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_68a81434ea40819cbce2a490383da69e014223f5b3c50db0', content=[ResponseOutputText(annotations=[], text="I don't have real-time data access to provide current weather conditions. However, Dubai typically has a desert climate, characterized by hot summers and mild winters. In summer (June to September), temperatures can exceed 40°C (104°F), while winter (December to February) sees milder temperatures, ranging from around 14°C to 24°C (57°F to 75°F) during the day.\n\nFor the most accurate and up-to-date weather information, I recommend checking a reliable weather website or app.", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')],
 'parallel_tool_calls': True,
 'temperature': 1

In [6]:
response.output[0].__dict__

{'id': 'msg_68a81434ea40819cbce2a490383da69e014223f5b3c50db0',
 'content': [ResponseOutputText(annotations=[], text="I don't have real-time data access to provide current weather conditions. However, Dubai typically has a desert climate, characterized by hot summers and mild winters. In summer (June to September), temperatures can exceed 40°C (104°F), while winter (December to February) sees milder temperatures, ranging from around 14°C to 24°C (57°F to 75°F) during the day.\n\nFor the most accurate and up-to-date weather information, I recommend checking a reliable weather website or app.", type='output_text', logprobs=[])],
 'role': 'assistant',
 'status': 'completed',
 'type': 'message'}

## Step 4: Define the input schema for our tool

Lets try giving our `get_weather` function as a tool to our AI.

A basic format of a tool schema is defined in the [OpenAI function calling docs](https://platform.openai.com/docs/guides/function-calling)

Step 1: Define the tool in a format OpenAI can understand

In [35]:
# Tool schema

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get the current temperature in Celsius for a given latitude and longitude",
    "parameters": {
        "type": "object",
        "properties": {
            "lat": {"type": "number"},
            "lon": {"type": "number"}
        },
        "required": ["lat", "lon"],
        "additionalProperties": False
    },
    "strict": True
}]

## Step 5: Pass the tool schema over to the model

In [38]:
# Pass in the function schema to our model

input_messages = [{"role":"user", "content":"What's the weather like in Dubai?"}]

response = client.responses.create(
    model="gpt-4o-mini",
    input=input_messages,
    tools=tools
)

In [39]:
# Print the response dict
response.__dict__

{'id': 'resp_68a830b2951c81a39c3495b37dd0b75c0f2075a84493bad4',
 'created_at': 1755852978.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [ResponseFunctionToolCall(arguments='{"lat":25.276987,"lon":55.296249}', call_id='call_iQfbtH8KavCRy736tXaEGbQX', name='get_weather', type='function_call', id='fc_68a830b3abd881a39233bbbe3838e5bf0f2075a84493bad4', status='completed')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'lat': {'type': 'number'}, 'lon': {'type': 'number'}}, 'required': ['lat', 'lon'], 'additionalProperties': False}, strict=True, type='function', description='Get the current temperature in Celsius for a given latitude and longitude')],
 'top_p': 1.0,
 'background': False,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'previous_response_id': N

In [40]:
response.output[0].__dict__

{'arguments': '{"lat":25.276987,"lon":55.296249}',
 'call_id': 'call_iQfbtH8KavCRy736tXaEGbQX',
 'name': 'get_weather',
 'type': 'function_call',
 'id': 'fc_68a830b3abd881a39233bbbe3838e5bf0f2075a84493bad4',
 'status': 'completed'}

## Step 6: Format the tool call response from the LLM

In [41]:
response.output[0]

ResponseFunctionToolCall(arguments='{"lat":25.276987,"lon":55.296249}', call_id='call_iQfbtH8KavCRy736tXaEGbQX', name='get_weather', type='function_call', id='fc_68a830b3abd881a39233bbbe3838e5bf0f2075a84493bad4', status='completed')

In [42]:
# Format tool call response
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

print("Tool Call: ", tool_call)
print("Arguments: ", args)

Tool Call:  ResponseFunctionToolCall(arguments='{"lat":25.276987,"lon":55.296249}', call_id='call_iQfbtH8KavCRy736tXaEGbQX', name='get_weather', type='function_call', id='fc_68a830b3abd881a39233bbbe3838e5bf0f2075a84493bad4', status='completed')
Arguments:  {'lat': 25.276987, 'lon': 55.296249}


## Step 7: Pass on the tool call arguments to our tool/python function

We now need to pass on the arguments received by the model to our python function or tool

In [43]:
# Pass tool call into python function
result = get_weather(args["lat"], args["lon"])

In [44]:
result

35.4

## Step 8: Append the response of the tool into the message list

In [49]:
# Convert tool call and response of tool to messages list

# input_messages.append(tool_call)
input_messages

[{'role': 'user', 'content': "What's the weather like in Dubai?"},
 ResponseFunctionToolCall(arguments='{"lat":25.276987,"lon":55.296249}', call_id='call_iQfbtH8KavCRy736tXaEGbQX', name='get_weather', type='function_call', id='fc_68a830b3abd881a39233bbbe3838e5bf0f2075a84493bad4', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_iQfbtH8KavCRy736tXaEGbQX',
  'output': '35.4'}]

In [48]:
input_messages.append({
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

## Step 9: Pass the message list into the model

In [50]:
input_messages

[{'role': 'user', 'content': "What's the weather like in Dubai?"},
 ResponseFunctionToolCall(arguments='{"lat":25.276987,"lon":55.296249}', call_id='call_iQfbtH8KavCRy736tXaEGbQX', name='get_weather', type='function_call', id='fc_68a830b3abd881a39233bbbe3838e5bf0f2075a84493bad4', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_iQfbtH8KavCRy736tXaEGbQX',
  'output': '35.4'}]

In [51]:
response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

The current temperature in Dubai is approximately 35.4°C. Let me know if you need more details or a weather forecast!


# Resources:

- [OpenAIs Function Calling Guide](https://platform.openai.com/docs/guides/function-calling?api-mode=responses)

## Your Turn: Build Your Own Custom Tool

Now that you've seen how to define a Python function as a tool and connect it to an LLM, it's time to get creative!

### Pick your challenge
**1. Wikipedia Article Summarizer**
- Fetch a wikipedia article you are interested in learning about and have an LLM summarize it
- [Wikipedia API](https://pypi.org/project/Wikipedia-API/)

**2. News Summarizer**
- Fetch latest headlines or news by topic.
- [NewsAPI](https://newsapi.org)
- [CurrentsAPI](https://currentsapi.services/en/docs/)

**3. Stock Market Data**
- Get real-time stock prices or company info
- [Alpha Vantage](https://www.alphavantage.co/)
- [yfinance](https://pypi.org/project/yfinance/)

**4. Movie Info**
- Get movie ratings, cast, plot summaries.
- [IMDb](https://imdbapi.dev/)
- [OMDb API](https://www.omdbapi.com/)

**5. NASA API**
- Astronomy facts, country data, etc.
- [NASA APIs](https://api.nasa.gov/)

**6. Your own custom tool**
- Think of a real-world use case where an LLM could benefit from calling a custom function.


---

**Tips**:
- Use clear function names and docstrings.
- Handle input arguments and outputs carefully.
- Print the LLM's tool call and your function's output in a readable way.

Be sure to place your submissions in `AI-Engineering-Intermediate/Part1/community-contributions`

I'm super excited to see what you come up with :D